In [ ]:
def Make_Question(n, m, factor=10, Density = 0.75, option = "1-norm", distance = 2):
    nx, ny = (n, m)
    x = np.arange(nx,dtype=float)
    y = np.arange(ny,dtype=float)
    xv, yv = np.meshgrid(x, y)
    Px = np.power(factor,-np.abs(xv-n//2))
    Py = np.power(factor,-np.abs(yv-m//2))
    mesh_porbability = np.multiply(Px,Py)
    final_probability = np.multiply(Px,Py)/np.sum(mesh_porbability)
    amount_panel = np.int(Density*n*m)
    allocation = np.random.choice(np.arange(n*m), amount_panel, replace=False, p = final_probability.flatten())
    x_position = allocation // m
    y_position = allocation %  m
    
    vertices = np.vstack((x_position, y_position)).T
    example = vertices[0]
    network = np.zeros((amount_panel,amount_panel))
    for i, vertex in enumerate(vertices):
        example = vertices[i]
        if option == "1-norm":
            # adjacent option
            index = np.where(np.sum(np.abs(vertices-example),axis = 1) == 1)
        elif option == "2-norm":
            # include diag or more
            index = np.where(np.linalg.norm(np.abs(vertices-example),axis = 1) < distance)
        else:
            print("error_input")
        network[i][index] = 1
    network_adv = np.triu(np.pad(network, ((1,0), (1,0)), 'constant', constant_values=(1, 1)),k=1)
    
    pos = {0: (n+0.7,m)} 
    for i, vertex in enumerate(vertices):
        temp_pos = (vertex[0],vertex[1])
        pos.update({i+1: temp_pos})
    return network_adv,pos

def complete_Graph (network,pos):
    # Draw the Directed Graph
    G = nx.DiGraph()
    G = nx.from_numpy_array(network)
    
    # Label Edge with following rules:
    # 1. num(BUS)<num(branch)
    # 2. priority(BUS)>priority(branch)
    edge_label = {}
    for j, edge in enumerate(G.edges()):
        edge_label.update({(edge[0],edge[1]): str(j)})
        
    nx.draw(G,pos,with_labels=True)
    a = nx.draw_networkx_edge_labels(G,pos,edge_labels = edge_label)
    return G, edge_label

def sub_Graph (network,pos,compelete_label):
    G_prime = nx.DiGraph()
    G_prime = nx.from_numpy_array(network)
    G_prime.remove_node(0)
    
    sub_edge_label = {}
    for j, edge in enumerate(G_prime.edges()):
        sub_edge_label.update({(edge[0],edge[1]): compelete_label[(edge[0],edge[1])]})
    
    nx.draw(G_prime,pos,with_labels=True)
    a = nx.draw_networkx_edge_labels(G_prime,pos,edge_labels = sub_edge_label)
    return G_prime, sub_edge_label

def edge_map (G, compelete_label, sub_edge_label):
    complete_edge = np.asarray(G.edges())
    new_map = np.zeros((len(compelete_label),len(compelete_label)))
    for edge in sub_edge_label:
    
        map_index_0, _ = np.where(complete_edge == edge[0])
        map_index_1, _ = np.where(complete_edge == edge[1])
    
        edge_index = np.int(compelete_label[(edge[0],edge[1])])
        new_map[edge_index][map_index_0] = 1
        new_map[edge_index][map_index_1] = 1
        
        new_map = new_map- np.diag(np.diag(new_map))
        
        lowest_edge = len(compelete_label)- len(sub_edge_label)
    return new_map, lowest_edge

#basic_cost shape => (1,number_of_edge)
# such as : basic_cost = np.vstack(np.ones(number_of_edge))

def Objective(pos,compelete_label,basic_cost,flow_cost = 1):
    #the set of capacity types
    Q = len(pos)-1  
    number_of_edge = len(compelete_label)
    QUBO_matrix = np.zeros((number_of_edge*Q,number_of_edge*Q))

    ##
    
    temp_1 = basic_cost*np.ones((number_of_edge,Q))
    term_1 = np.diag(temp_1.flatten())

    ##
    temp_2 = np.multiply(np.ones((number_of_edge,Q)),flow_cost*np.arange(Q))
    term_2 = np.diag(temp_2.flatten())

    #objective
    QUBO_matrix = term_1 + term_2
    return QUBO_matrix

def Constraint_1(pos,compelete_label,Final_map,penalty = 50):
    # constraint 1
    # QUBO matrix = 2 sum{m,i \in leaving edge}( sum{n,j \in Q}( x_ij*x_mn))     [term 2]
    #             - 3 sum{e \in leaving edge}( sum{d \in Q}( x_ed))              [term 1]
    #             + 1                                                            [term 3]
    #Kick the node 0 from the set of capacity since it does not generate energy
    Q = len(pos)-1  
    number_of_edge = len(compelete_label)
    
    #construct 2 terms independently
    QUBO_matrix = np.diag(np.zeros(number_of_edge * Q))
    for edge_out in Final_map:
        index = np.hstack(np.asarray(np.where(edge_out)))
        if len(index) == 0:
            continue
    
        #QUBO＿index = Q * quotient(index) + remainder (q) 
        q = np.arange(Q)
        QUBO_index = np.outer(index ,Q * np.ones(Q)) +  q
        QUBO_index_flatten = QUBO_index.reshape(1,-1)

        # Generate corresponding index mesh grid
        QUBO_index_x,QUBO_index_y = np.meshgrid(QUBO_index_flatten,QUBO_index_flatten)
    
        QUBO_matrix_index_term_2 = np.concatenate((QUBO_index_x.reshape(-1,1), QUBO_index_y.reshape(-1,1)), axis=1)
        QUBO_matrix_index_term_1 = np.concatenate((QUBO_index_flatten.reshape(-1,1), QUBO_index_flatten.reshape(-1,1)), axis=1)
    
    
        # term 1
        for indices in QUBO_matrix_index_term_1:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] -= 2
    
        # term 2
        for indices in QUBO_matrix_index_term_2:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] += 1
    
        # term 3
        QUBO_matrix += 1
    return QUBO_matrix * penalty 

def Constraint_2(pos,compelete_label,Final_map,penalty = 50):
    # constraint 2 
    # QUBO matrix = (sum{e \in leaving edge}( sum{d \in Q}(d * x_ed))   - sum{e \in enter edge}( sum{d \in Q}(d * x_ed)) - 1)^2
    #             = (A-B-1)^2
    #             =(A-1)^2 +(B-1)^2 -2AB + 4B - 1
    #Kick the node 0 from the set of capacity since it does not generate energy
    Q = len(pos)-1  
    number_of_edge = len(compelete_label)
    #construct  terms independently
    # constraint 2 A
    QUBO_matrix = np.diag(np.zeros(number_of_edge*Q))

    for edge_out in Final_map:
        index_out = np.hstack(np.asarray(np.where(edge_out)))
        if len(index_out) == 0:
            continue
        q = np.arange(Q)

        #QUBO＿index = Q * quotient(index) + remainder (q) 
        QUBO_index_out = np.outer(index_out ,Q * np.ones(Q)) +  q
        QUBO_index_out_flatten = QUBO_index_out.reshape(1,-1)

        #print(QUBO_index)
        # Generate corresponding index mesh grid
        QUBO_index_out_x,QUBO_index_out_y = np.meshgrid(QUBO_index_out_flatten,QUBO_index_out_flatten)

        QUBO_matrix_index_out_term_1 = np.concatenate((QUBO_index_out_x.reshape(-1,1), QUBO_index_out_y.reshape(-1,1)), axis=1)
        QUBO_matrix_index_out_term_2 = np.concatenate((QUBO_index_out_flatten.reshape(-1,1), QUBO_index_out_flatten.reshape(-1,1)), axis=1)


        # term (A-1)_1
        for indices in QUBO_matrix_index_out_term_1:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] += (indices[0].astype(int)%Q)*(indices[1].astype(int)%Q)

        # term (A-1)_2
        for indices in QUBO_matrix_index_out_term_2:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] -= 2*(indices[0].astype(int)%Q)
        # term (A-1)_3

        QUBO_matrix += 1


    for edge_in in Final_map.T:
        index_in = np.hstack(np.asarray(np.where(edge_in)))
        if len(index_in) == 0:
            continue
        q = np.arange(Q)

        #QUBO＿index = Q * quotient(index) + remainder (q) 
        QUBO_index_in = np.outer(index_in ,Q * np.ones(Q)) +  q
        QUBO_index_in_flatten = QUBO_index_in.reshape(1,-1)

        #print(QUBO_index_in)

        # Generate corresponding index mesh grid
        QUBO_index_in_x,QUBO_index_in_y = np.meshgrid(QUBO_index_in_flatten,QUBO_index_in_flatten)

        QUBO_matrix_index_in_term_1 = np.concatenate((QUBO_index_in_x.reshape(-1,1), QUBO_index_in_y.reshape(-1,1)), axis=1)
        QUBO_matrix_index_in_term_2 = np.concatenate((QUBO_index_in_flatten.reshape(-1,1), QUBO_index_in_flatten.reshape(-1,1)), axis=1)
        #print(edge_out,index)
        #print(index)
        #print(np.outer(Q * np.ones(Q),q))
        #print(QUBO_matrix_index_term_2)
        #print(QUBO_matrix_index_term_1)

        # term (B-1)_1
        for indices in QUBO_matrix_index_in_term_1:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] += (indices[0].astype(int)%Q)*(indices[1].astype(int)%Q)

        # term (B-1)_2
        for indices in QUBO_matrix_index_in_term_2:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] -= 2*(indices[0].astype(int)%Q)

        # term (B-1)_3 + last term (-1)

        # QUBO_matrix += 1-1  (zero)

        # term 4B

        for indices in QUBO_matrix_index_out_term_2:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] += 4*(indices[0].astype(int)%Q)


    # term 2AB
    for i in range(len(Final_map)):
        index_x = np.hstack(np.asarray(np.where(Final_map[i,:])))
        index_y = np.hstack(np.asarray(np.where(Final_map[:,i])))
        if len(index_x)*len(index_y) == 0:
            continue
        q = np.arange(Q)
        #print(index_x,index_y)

        #QUBO＿index = Q * quotient(index) + remainder (q) 
        QUBO_index_x = np.outer(index_x ,Q * np.ones(Q)) +  q
        QUBO_index_x_flatten = QUBO_index_x.reshape(1,-1)

        #QUBO＿index = Q * quotient(index) + remainder (q) 
        QUBO_index_y = np.outer(index_y ,Q * np.ones(Q)) +  q
        QUBO_index_y_flatten = QUBO_index_y.reshape(1,-1)

        #print( QUBO_index_x_flatten,  QUBO_index_y_flatten)

        QUBO_index_x,QUBO_index_y = np.meshgrid(QUBO_index_x_flatten,QUBO_index_y_flatten)

        QUBO_matrix_index_xy = np.concatenate((QUBO_index_x.reshape(-1,1), QUBO_index_y.reshape(-1,1)), axis=1)

        #print(QUBO_matrix_index_xy)
        for indices in QUBO_matrix_index_xy:
            QUBO_matrix[indices[0].astype(int)][indices[1].astype(int)] -= 2*(indices[0].astype(int)%Q)*(indices[1].astype(int)%Q)
    
        
        
    return QUBO_matrix * penalty 

def capacity_subtree_limit_generator(pos,lowest_edge_number):
    # the max and min number of subtrees with a capacity equal to t in T 
    # option 1 (disagrd)
    #m_t_u = np.round(0.55 * Q * np.ones(Q))
    #m_t_l = np.round(0.25 * Q * np.ones(Q))
    
    Q = len(pos)-1
    
    #应该作为 Input 
    m_t_u = np.random.randint(1.2*Q, size=Q)
    m_t_l = np.random.randint(Q, size=Q)-np.ceil(Q/2)
    
    m_t_u_update = np.where(m_t_u>=lowest_edge_number, lowest_edge_number,  m_t_u)
    m_t_l_update = np.where(m_t_l<=0, 0,  m_t_l)
    
    return m_t_u_update,m_t_l_update

def Constraint_3_part_1(pos,compelete_label,lowest_edge_number,QUBO_matrix,m_t_u,penalty = 50):
    #enegy capacity
    Q = len(pos)-1 
    number_of_edge = len(compelete_label)
    
    #
    Bus_edge = lowest_edge_number

    disturbance =0.0001  #optinal solution for -inf happens log(0) = -inf log(0.00001)=-13
    
    #determine how many slack Vars we need 
    svn_up = np.ceil(np.log2(Bus_edge-m_t_u + disturbance))
    
    #the place with positive numbers need slack var will stay, the rest will be 0
    svn_up_update = np.where(svn_up>=0, svn_up,  0)

    #reate a blank QUBO table (optional)
    #QUBO_matrix = np.zeros_like(QUBO_matrix)


    ## (sum x_et)^2 - 2m_t^u sum(x_et) 
    
    ## compute (sum x_et)^2
    # make the a single period 
    skeleton = np.zeros(Q)
    skeleton[0] = 1
    # For t, the (sum x_et)^2 only shows 1 when they shared same t
    # 1 appears periodically each line with period of Q  (Example: Q = 4 edge = 3 X_{e*Q + t})
    # each line will copy the array above and do a right shift 
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    skeleton_x = np.tile(skeleton ,Bus_edge )
    
    #create a blank QUBO table
    skeleton_final = np.zeros((Bus_edge*Q,Bus_edge*Q))
    
    # do right shift one lane by one lane
    for i in range(Bus_edge*Q):
        skeleton_final[i,:] = skeleton_x
        skeleton_x = np.roll(skeleton_x, 1)
        

    # compute 2m_t^u sum(x_et) 
    skeleton_final -= np.diag(2*np.tile(m_t_u ,Bus_edge))
    QUBO_matrix[:Bus_edge*Q,:Bus_edge*Q] += skeleton_final


    ## (sum x_ti)^2 - 2^(i+1) m_t^u sum(x_it)
    # Since x_it is not useful in QUBO, in order to make our life easy, we change our way to index our x_it by 
    # switching the index of x_it from x_it to x_ti

    QUBO_matrix_new = QUBO_matrix  
    # Need to expand the Matrix with a order like (ex Q = 1,slack = 2)
    #          x0t|x1t|
    #    *|*|*|
    #    *|*|*|
    #    *|*|*|
    #x0t| 
    #x1t|
    # start rolling from t = 0 to t = Q-1 (size = Q) 
    for t in range(Q):
        # if we dont need slack var just pass and move on
        if np.int(svn_up_update[t])==0:
            continue
        # padding the Right-bottom for xit for each t. the size of intersection in each t should be 
        #(svn_up_update[t],svn_up_update[t]) 
        QUBO_matrix_new = np.pad(QUBO_matrix_new, ((0, np.int(svn_up_update[t])), (0, np.int(svn_up_update[t]))), 'constant', constant_values=0)
        
        # set 2^(i+j) in a grid-like shape
        coeff_set = np.power(2,np.arange(svn_up_update[t]))
        slack_coeff = np.outer(coeff_set,coeff_set)
        
        
        # (sum 2^(i+j) x_it)^2
        QUBO_matrix_new[-np.int(svn_up_update[t]):,-np.int(svn_up_update[t]):] += slack_coeff
        # 2^(i+1) m_t^u sum(x_it)
        QUBO_matrix_new[-np.int(svn_up_update[t]):,-np.int(svn_up_update[t]):] -= 2*np.diag(coeff_set)*m_t_u[t]

        ## 2^(i+1) x'_it x_et + (m_t^u)^2 
        # locate the intersection between x_et and x_it 
        # the new sequence of padding area is x_1t,x_2t,x_3t
        
        # the corresponding x_et should be index = e*Q+t   e \in E\E'(edges connect to the node that collecting the power)
        index_bus_slack = np.arange(Bus_edge)*Q + t
        
        #prepare the side matrix 
        # where does x'_it x_et locate ?
        #    (ex Q = 1,slack = 2, p \in E\E')
        #      E\E'      x0t|x1t|
        #     o | o | * |2^0|2^1| Matrix 1
        #E\E' o | o | * |2^0|2^1| 
        #     * | * | * |   |   | 
        #x0t 2^0|2^0|   |   |   |
        #x1t 2^1|2^1|   |   |   |
        #    Matrix 2
        # it is ez to find that Matrix_1 = Matrix_2.T (Since the whole matrix  is symmetric right now)
        # we can take advantage of it
        # set up 2 ^ i i = 0,1 ... svn_up_update[t]-1
        substitution = 2 * np.power(2,np.arange(svn_up_update[t]))
        
        # set up matrix 1 with shape(Bus_edge , svn_up_update[t])
        substitution_v = np.tile(substitution,(Bus_edge,1))
        
        # et it will have intersection only when they shared same t
        # we can find such index(x_et) by  
        #x_index = e*Q+t 
        #y_index = i     (AKA, last svn_up_update[t] columns/rows)
        substitution_h = substitution_v.T
        QUBO_matrix_new[index_bus_slack,-np.int(svn_up_update[t]):] = substitution_v

        # Since they are is symmetric. we can just do copy, tranpose, and paste to set up Matrix_2
        QUBO_matrix_new[-np.int(svn_up_update[t]):,index_bus_slack] = substitution_h

    # 存疑  (Questionable)  应该加到T类上，还是全加？
    QUBO_matrix_new += np.dot(m_t_u.T,m_t_u)
    
    
    return QUBO_matrix_new*penalty

def Constraint_3_part_2(pos,compelete_label,lowest_edge_number,QUBO_matrix,m_t_l,penalty = 50):
    #enegy capacity
    Q = len(pos)-1 
    number_of_edge = len(compelete_label)
    
    #
    Bus_edge = lowest_edge_number

    disturbance =0.0001  #optinal solution for -inf happens log(0) = -inf log(0.00001)=-13
    
    #determine how many slack Vars we need 
    svn_up = np.ceil(np.log2(Bus_edge-m_t_l + disturbance))
    
    #the place with positive numbers need slack var will stay, the rest will be 0
    svn_up_update = np.where(svn_up > 0, svn_up,  0)

    #reate a blank QUBO table (optional)
    #QUBO_matrix = np.zeros_like(QUBO_matrix)


    ## (sum x_et)^2 - 2m_t^u sum(x_et) 
    
    ## compute (sum x_et)^2
    # make the a single period 
    skeleton = np.zeros(Q)
    skeleton[0] = 1
    # For t, the (sum x_et)^2 only shows 1 when they shared same t
    # 1 appears periodically each line with period of Q  (Example: Q = 4 edge = 3 X_{e*Q + t})
    # each line will copy the array above and do a right shift 
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    # 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 |
    # 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 |
    # 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 |
    # 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 |
    skeleton_x = np.tile(skeleton ,Bus_edge )
    
    #create a blank QUBO table
    skeleton_final = np.zeros((Bus_edge*Q,Bus_edge*Q))
    
    # do right shift one lane by one lane
    for i in range(Bus_edge*Q):
        skeleton_final[i,:] = skeleton_x
        skeleton_x = np.roll(skeleton_x, 1)
        

    # compute 2m_t^l sum(x_et) 
    skeleton_final -= np.diag(2*np.tile(m_t_l ,Bus_edge))
    QUBO_matrix[:Bus_edge*Q,:Bus_edge*Q] += skeleton_final


    ## (sum x_it)^2 + 2^(i+1) m_t^l sum(x_it)
    # Since x_it is not useful in QUBO, in order to make our life easy, we change our way to index our x_it by 
    # switching the index of x_it from x_it to x_ti

    QUBO_matrix_new = QUBO_matrix  
    # Need to expand the Matrix with a order like (ex Q = 1,slack = 2)
    #          x0t|x1t|
    #    *|*|*|
    #    *|*|*|
    #    *|*|*|
    #x0t| 
    #x1t|
    # start rolling from t = 0 to t = Q-1 (size = Q) 
    for t in range(Q):
        # if we dont need slack var just pass and move on
        if np.int(svn_up_update[t])==0:
            continue
        # padding the Right-bottom for xit for each t. the size of intersection in each t should be 
        #(svn_up_update[t],svn_up_update[t]) 
        QUBO_matrix_new = np.pad(QUBO_matrix_new, ((0, np.int(svn_up_update[t])), (0, np.int(svn_up_update[t]))), 'constant', constant_values=0)
        
        # set 2^(i+j) in a grid-like shape
        coeff_set = np.power(2,np.arange(svn_up_update[t]))
        slack_coeff = np.outer(coeff_set,coeff_set)
        
        
        # (sum 2^(i+j) x_it)^2
        QUBO_matrix_new[-np.int(svn_up_update[t]):,-np.int(svn_up_update[t]):] += slack_coeff
        #- 2m_t^u sum(x_it)
        QUBO_matrix_new[-np.int(svn_up_update[t]):,-np.int(svn_up_update[t]):] += 2*np.diag(coeff_set)*m_t_l[t]

        ## -2^(i+1) x'_it x_et + (m_t^u)^2 
        # locate the intersection between x_et and x_it 
        # the new sequence of padding area is x_1t,x_2t,x_3t
        
        # the corresponding x_et should be index = e*Q+t   e \in E\E'(edges connect to the node that collecting the power)
        index_bus_slack = np.arange(Bus_edge)*Q + t
        
        #prepare the side matrix 
        # where does x'_it x_et locate ?
        #    (ex Q = 1,slack = 2, p \in E\E')
        #      E\E'      x0t|x1t|
        #     o  | o  | * |-2^0|-2^1| Matrix 1
        #E\E' o  | o  | * |-2^0|-2^1| 
        #     *  | *  | * |    |    | 
        #x0t -2^0|-2^0|   |    |    |
        #x1t -2^1|-2^1|   |    |    |
        #    Matrix 2
        # it is ez to find that Matrix_1 = Matrix_2.T (Since the whole matrix  is symmetric right now)
        # we can take advantage of it
        # set up 2 ^ i i = 0,1 ... svn_up_update[t]-1
        substitution = 2 * np.power(2,np.arange(svn_up_update[t]))
        
        # set up matrix 1 with shape(Bus_edge , svn_up_update[t])
        substitution_v = np.tile(substitution,(Bus_edge,1))
        
        # et it will have intersection only when they shared same t
        # we can find such index(x_et) by  
        #x_index = e*Q+t 
        #y_index = i     (AKA, last svn_up_update[t] columns/rows)
        substitution_h = substitution_v.T
        QUBO_matrix_new[index_bus_slack,-np.int(svn_up_update[t]):] = -substitution_v

        # Since they are is symmetric. we can just do copy, tranpose, and paste to set up Matrix_2
        QUBO_matrix_new[-np.int(svn_up_update[t]):,index_bus_slack] = -substitution_h

    # 存疑  (Questionable)  应该加到T类上，还是全加？
    QUBO_matrix_new += np.dot(m_t_l.T,m_t_l)
    
    return QUBO_matrix_new*penalty

# Ban_map size(edge(E\E'),Q) Full Binary map
def Constraint_4(lowest_edge_number,Ban_map,penalty = 1000):
    Ban_map_flatten = Ban_map.flatten()
    
    penalty_carrier = np.diag(Ban_map_flatten)
    # return a partial penalty matrix with size (e (\in E\E') *Q, e (\in E\E') *Q)
    
    return penalty*penalty_carrier 


network,pos = Make_Question(3,3,Density = 0.85, option = "1-norm", distance = 2)
G, compelete_label = complete_Graph(network,pos)
G_prime, sub_edge_label = sub_Graph (network,pos,compelete_label)
Final_map, lowest_edge_number = edge_map(G, compelete_label, sub_edge_label)

basic_cost = np.vstack(np.ones(len(compelete_label)))

QUBO_Obj = Objective(pos,compelete_label,basic_cost,flow_cost = 1)

QUBO_Constraint_1 = Constraint_1(pos,compelete_label,Final_map,penalty = 50)

QUBO_Constraint_2 = Constraint_2(pos,compelete_label,Final_map,penalty = 50)

QUBO_matrix = QUBO_Obj + QUBO_Constraint_1 + QUBO_Constraint_2

print(np.shape(QUBO_matrix ))

m_t_u,m_t_l = capacity_subtree_limit_generator(pos,lowest_edge_number)

QUBO_matrix_1 = Constraint_3_part_1(pos,compelete_label,lowest_edge_number,QUBO_matrix,m_t_u,penalty = 50)
QUBO_matrix_2 = Constraint_3_part_2(pos,compelete_label,lowest_edge_number,QUBO_matrix_1,m_t_l,penalty = 50)

Ban_map = np.zeros((lowest_edge_number*(len(pos)-1)))

QUBO_Constraint_4 = Constraint_4(lowest_edge_number,Ban_map,penalty = 100)

QUBO_matrix_2[:lowest_edge_number*(len(pos)-1),:lowest_edge_number*(len(pos)-1)] +=  QUBO_Constraint_4

Final_QUBO_matrix = QUBO_matrix_2

print(np.shape(Final_QUBO_matrix))